In [72]:
from lxml import etree as ET #lxml is faster
import glob #to use multiple files
from collections import Counter #to count unique words

Dataset

In [93]:
#one file

tree = ET.parse('dataset/A1E.xml')
root = tree.getroot()
# print(root)

text=[]

for s in root.iter('s'):
    text.append('<s>')
    for elem in s.findall('*'):
        text.append(elem.text)
    text.append('</s>')

print("Word count: ",len(text))


Word count:  11616


In [94]:
#all files

files = glob.glob('fulldataset/*.xml')

textfull = []

for fileName in files:
    
    tree = ET.parse(fileName)
    root = tree.getroot()

    for s in root.iter('s'):
        textfull.append('<s>')
        for elem in s.findall('*'):
            textfull.append(elem.text)
        textfull.append('</s>')
    
print("Word count: ",len(textfull))

Word count:  5392173


unqiue words and their count

In [95]:
uniqueWords = {}

for word in text:
    if word in uniqueWords:
        uniqueWords[word] += 1
    else:
        uniqueWords[word] = 1

# for word, count in uniqueWords.items():
#     print(f"{word}: {count}")

ngram function

In [96]:
def ngram(text, n): #n is size of ngrams

    ngramCounts = {} #dict to store ngrams

    for i in range(len(text)-n+1): #len(text)-n+1 is the number of all possible ngrams

        ngram = tuple(text[i:i+n]) #creates ngram

        if ngram in ngramCounts: #check to see if it already exists
            ngramCounts[ngram] += 1
        else:
            ngramCounts[ngram] = 1

    return ngramCounts

In [97]:
def ngramCounter(text, n): #ngram function using counter
    
    ngramCounts = Counter()

    for i in range(len(text)-n+1):
        ngram = tuple(text[i:i+n])
        ngramCounts[ngram] += 1
    
    return ngramCounts

Testing ngram function

In [108]:
teststring = '<s> I am Sam </s> <s> Sam I am </s> <s> I do not like green eggs and ham </s>'
testlist = list(teststring.split(" "))

# print("Unigram:\n",ngram(testlist,1), "\n")
# print("Bigram:\n",ngram(testlist,2), "\n")
# print("Trigram:\n",ngram(testlist,3))

ngram probabilities

In [104]:
def calcNGramProbabilities(text, n):
    ngrams = ngram(text, n)

    totalNgrams = len(text) - n + 1
    
    ngramProbs = {ngram: count / totalNgrams for ngram, count in ngrams.items()}
    
    # ngramProbs = 
    
    return ngramProbs

In [110]:
calcNGramProbabilities(testlist,2)

{('<s>', 'I'): 0.10526315789473684,
 ('I', 'am'): 0.10526315789473684,
 ('am', 'Sam'): 0.05263157894736842,
 ('Sam', '</s>'): 0.05263157894736842,
 ('</s>', '<s>'): 0.10526315789473684,
 ('<s>', 'Sam'): 0.05263157894736842,
 ('Sam', 'I'): 0.05263157894736842,
 ('am', '</s>'): 0.05263157894736842,
 ('I', 'do'): 0.05263157894736842,
 ('do', 'not'): 0.05263157894736842,
 ('not', 'like'): 0.05263157894736842,
 ('like', 'green'): 0.05263157894736842,
 ('green', 'eggs'): 0.05263157894736842,
 ('eggs', 'and'): 0.05263157894736842,
 ('and', 'ham'): 0.05263157894736842,
 ('ham', '</s>'): 0.05263157894736842}